## Improting data:

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Data/Stocks/a.us.txt')

In [ ]:
print('starting date: ',df['Date'][0])
print('Last date: ',df['Date'][len(df)-1])
print('size of data: ', len(df))

## Since our data is from 1999 - 2011 which is quite big so, I'll slice data to make it smaller from 2010 - 2017

In [ ]:
df = df[df['Date']> '2010-1-1']

# Setiing date as index:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(df['Date'], inplace=True)
df.drop(['Date','High','Low','Open','OpenInt','Volume'], axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,6))
plt.plot(df['Close'])
plt.title('Closing price of stock')

In [ ]:
# Function for splitting the data:

def create_shape(df, timesteps):
    X,y = [], []
    for i in range(timesteps, len(df)):
        X.append(df[i-timesteps:i,0])
        y.append(df[i,0])
    return np.array(X), np.array(y)

### Normalizing data and splitting in train and test set.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0,1))

scaled_df = scaler.fit_transform(df)
train = scaled_df[:-12] # train_set (7 + time_steps)
test = scaled_df[-12:] # test_set

X_train, y_train=create_shape(train, 5)
# Creating shape(batch_size, time_steps, 1) for lstm model:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1],1)

X_test, y_test=create_shape(test, 5)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1],1)    

## Creating LSTM network:

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))

model.add(Dropout(0.1)) 
model.add(LSTM(50))

model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer= 'adam',metrics=['mean_absolute_error'])
model.summary()

## Fitting the network on training set:

In [ ]:
model.fit(X_train,y_train,epochs=90, verbose=0)

## Making prediction for next 7 days:

In [ ]:
prediction = model.predict(X_test)
prediction_inv = scaler.inverse_transform(prediction)
prediction_inv

## Inverse transforming the y_test using minmax scaler to compare with predictions:

In [ ]:
y_test_inv = y_test.reshape(y_test.shape[0], 1)
y_test_inv = scaler.inverse_transform(y_test_inv)

In [ ]:
from sklearn.metrics import mean_squared_error
print('-'*30,'\n',
      'mean square error:',mean_squared_error(prediction_inv,y_test_inv),
      '\n','-'*30)
plt.figure(figsize=(8,5))
plt.plot(prediction_inv)
plt.plot(y_test_inv)
plt.title('Predicted vs Actual')
plt.legend(['predicted','Actual'])